In [1]:
# Written by Adrianna Loback - 2019

import os
import json
import numpy as np
from core.data_processor_gs import DataLoader
from core.model import create_model
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
import pdb

Using TensorFlow backend.


In [2]:
configs = json.load(open('config_daily.json', 'r'))

In [3]:
# -- Data preparation:
data = DataLoader(os.path.join('../data', configs['data']['filename']), 
                  configs['data']['columns'])

# Load input (X) and target values (Y)
X, Y = data.get_train_data(
       seq_len=configs['data']['sequence_length'],
       normalise=configs['data']['normalise'])

In [4]:
# -- Implement grid search for hyperparam optimisation for CNN model:
# Fix random seed for reproducibility
seed = 3
np.random.seed(seed)

# Create model
model = KerasRegressor(build_fn=create_model)

In [5]:
# Define the grid search parameters
filts        = [32, 64, 128]
k_size       = [2]
pool_size    = [2]
neurons      = [50, 100, 150, 200]
optimizer    = ['SGD', 'Adam', 'Adadelta']
activation   = ['relu', 'sigmoid', 'linear']
dropout_rate = [0.2, 0.3]
batch_size   = [5, 10, 20]
epochs       = [1]

In [6]:
param_grid   = dict(filts=filts, k_size=k_size, pool_size=pool_size,
                    neurons=neurons, optimizer=optimizer, activation=activation,
                    dropout_rate=dropout_rate, batch_size=batch_size, epochs=epochs)

In [7]:
# Implement grid search (use all cores on local machine)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, Y)

Epoch 1/1
255/255 [==============================] - 0s 1ms/step - loss: 7.8123e-05


In [8]:
# Summarise results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means  = grid_result.cv_results_['mean_test_score']
stds   = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.000038 using {'activation': 'linear', 'batch_size': 5, 'dropout_rate': 0.2, 'epochs': 1, 'filts': 64, 'k_size': 2, 'neurons': 150, 'optimizer': 'Adam', 'pool_size': 2}
-0.000076 (0.000027) with: {'activation': 'relu', 'batch_size': 5, 'dropout_rate': 0.2, 'epochs': 1, 'filts': 32, 'k_size': 2, 'neurons': 50, 'optimizer': 'SGD', 'pool_size': 2}
-0.000067 (0.000046) with: {'activation': 'relu', 'batch_size': 5, 'dropout_rate': 0.2, 'epochs': 1, 'filts': 32, 'k_size': 2, 'neurons': 50, 'optimizer': 'Adam', 'pool_size': 2}
-0.000052 (0.000008) with: {'activation': 'relu', 'batch_size': 5, 'dropout_rate': 0.2, 'epochs': 1, 'filts': 32, 'k_size': 2, 'neurons': 50, 'optimizer': 'Adadelta', 'pool_size': 2}
-0.000070 (0.000021) with: {'activation': 'relu', 'batch_size': 5, 'dropout_rate': 0.2, 'epochs': 1, 'filts': 32, 'k_size': 2, 'neurons': 100, 'optimizer': 'SGD', 'pool_size': 2}
-0.000057 (0.000010) with: {'activation': 'relu', 'batch_size': 5, 'dropout_rate': 0.2, 'epochs': 1, 'fi